# LangChain: Models, Prompts and Output Parsers


## Outline

 * Direct API calls to OpenAI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

## Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [11]:
#!pip install python-dotenv
#!pip install openai

In [1]:
# import os
# import openai

from dotenv import load_dotenv # , find_dotenv

# This version uses find_dotenv() to get the .env file location
# But the actual implementation calls find_dotenv() by default when a path is not informed
#_ = load_dotenv(find_dotenv()) # read local .env file
_ = load_dotenv()

# This versions uses a global veriable to keep the api_key
# But the actual implementation gets the API key value directly from the system environment
#openai.api_key = os.environ['OPENAI_API_KEY']

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
# This code is not necessary anymore
# current_date = 2024-10-21
# target_date  = 2024-06-12

# [DEPRECATED]

# account for deprecation of LLM model
#import datetime
# Get the current date
#current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
#target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
#if current_date > target_date:
#    llm_model = "gpt-3.5-turbo"
#else:
#    llm_model = "gpt-3.5-turbo-0301"


In [3]:
# [NEW VERSION]

# Today: 2024-10-21
# The gpt-3.5-turbo currently points to gpt-3.5-turbo-0125.

llm_model = "gpt-3.5-turbo"

## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [4]:
# APIRemovedInV1: 
# You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.
# You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 
# Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`
# A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742

# [DEPRECATED]

#def get_completion(prompt, model=llm_model):
#    messages = [{"role": "user", "content": prompt}]
#    response = openai.ChatCompletion.create(
#        model=model,
#        messages=messages,
#        temperature=0, 
#    )
#    return response.choices[0].message["content"]

In [5]:
# [NEW VERSION]

from openai import OpenAI
client = OpenAI()

def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, 
    )   
    return response.choices[0].message.content

# The response structure:
#
# ChatCompletion(
#     id='chatcmpl-AKpBgYf0eJGOVIatmCJ46EcruB8L7', 
#     choices=[
#         Choice(
#             finish_reason='stop', 
#             index=0, logprobs=None,
#             message=ChatCompletionMessage(
#                 content='1+1 equals 2.', 
#                 refusal=None, 
#                 role='assistant', 
#                 audio=None, 
#                 function_call=None, 
#                 tool_calls=None
#             )
#         )
#     ], 
#     created=1729526688, 
#     model='gpt-3.5-turbo-0125', 
#     object='chat.completion', 
#     service_tier=None, 
#     system_fingerprint=None, 
#     usage=CompletionUsage(
#         completion_tokens=7, 
#         prompt_tokens=14, 
#         total_tokens=21, 
#         completion_tokens_details=CompletionTokensDetails(
#             audio_tokens=None, 
#             reasoning_tokens=0
#         ), 
#         prompt_tokens_details=PromptTokensDetails(
#             audio_tokens=None, 
#             cached_tokens=0
#         )
#     )
# )

In [6]:

get_completion("What is 1+1?")

# '1+1 equals 2.'

'1+1 equals 2.'

In [7]:
# Use Case: Treat emails sent by customers
# - Convert the message to American English, in a calm and respectful version

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

print(customer_email)

# 
# Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
# 


Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!



In [8]:
style = "American English in a calm and respectful tone"

print(style)

# American English in a calm and respectful tone

American English in a calm and respectful tone


In [9]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

# Translate the text that is delimited by triple backticks 
# into a style that is American English in a calm and respectful tone. 
# text: ```
# Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
# ```

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [10]:
response = get_completion(prompt)

In [11]:
response

# 'Arrr, I am quite upset that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty does not cover the cost of cleaning up my kitchen. I could really use your help right now, friend!'

"I am really frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, friend."

## Chat API : LangChain

Let's try how we can do the same using LangChain.

In [12]:
#!pip install --upgrade langchain

### Model

In [13]:
# LangChainDeprecationWarning: 
#   The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. 
#   An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. 
#   To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
#
#     chat = ChatOpenAI(temperature=0.0, model=llm_model)

# [DEPRECATED]

#from langchain.chat_models import ChatOpenAI

In [14]:
# [NEW VERSION]

from langchain_openai import ChatOpenAI

In [15]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0, model=llm_model)
chat

# ChatOpenAI(
#     client=<openai.resources.chat.completions.Completions object at 0x110d23410>, 
#     async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x110ee4da0>, 
#     root_client=<openai.OpenAI object at 0x110c08b30>, 
#     root_async_client=<openai.AsyncOpenAI object at 0x110c08a40>, 
#     temperature=0.0, 
#     model_kwargs={}, 
#     openai_api_key=SecretStr('**********')
# )

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10e9983b0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10e999d30>, root_client=<openai.OpenAI object at 0x10db8d1f0>, root_async_client=<openai.AsyncOpenAI object at 0x10e9983e0>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

### Prompt template

In [16]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}.
text: ```{text}```"""

print(template_string)

# Translate the text that is delimited by triple backticks into a style that is {style}.
# text: ```{text}```

Translate the text that is delimited by triple backticks into a style that is {style}.
text: ```{text}```


In [17]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [18]:
prompt_template.messages[0].prompt

# PromptTemplate(
#     input_variables=['style', 'text'], 
#     input_types={}, 
#     partial_variables={}, 
#     template='Translate the text that is delimited by triple backticks into a style that is {style}.\ntext: ```{text}```')

PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}.\ntext: ```{text}```')

In [19]:
prompt_template.messages[0].prompt.input_variables

# ['style', 'text']

['style', 'text']

In [20]:
customer_style = "American English in a calm and respectful tone"

print(customer_style)

# American English in a calm and respectful tone

American English in a calm and respectful tone


In [21]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

print(customer_email)

# Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!


Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!



In [22]:
customer_messages = prompt_template.format_messages(
    style=customer_style,
    text=customer_email)

In [23]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [24]:
print(customer_messages[0].content)

# Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone.
# text: ```
# Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
# ```

Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```


In [25]:
# LangChainDeprecationWarning: 
#   The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
#
#     customer_response = chat(customer_messages)

# [DEPRECATED]

# Call the LLM to translate to the style of the customer message
#customer_response = chat(customer_messages)

In [26]:
# [NEW VERSION]

customer_response = chat.invoke(customer_messages)

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('400 Client Error: Bad Request for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Empty request"}')


In [27]:
print(customer_response.content)

# I am really frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! 
# And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. 
# I could really use your help right now, friend.

I am really frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, friend.


In [28]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [29]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [31]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

# Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate.
# text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
# ```

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate.
text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```


In [32]:
service_response = chat.invoke(service_messages)
print(service_response.content)

# Ahoy there, valued customer! 
# The warranty be not coverin' the cost o' cleanin' yer galley because 'tis yer own fault fer misusin' yer blender by forgettin' to put the lid on afore startin' it. 
# 'Tis a tough break, matey! Fare thee well!

Ahoy there, me hearty customer! The warranty be not coverin' the cleanin' expenses for yer galley because 'tis yer own fault that ye misused yer blender by forgettin' to put the lid on afore startin' the blender. Tough luck! Farewell and may fair winds be with ye!


## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [33]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

# {'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [34]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [35]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

# input_variables=['text'] 
# input_types={} 
# partial_variables={} 
# messages=[
#     HumanMessagePromptTemplate(
#         prompt=PromptTemplate(
#             input_variables=['text'], 
#             input_types={}, 
#             partial_variables={}, 
#             template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'
#         ), 
#         additional_kwargs={}
#     )
# ]

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})]


In [36]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0, model=llm_model)
response = chat.invoke(messages)
print(response.content)

# {
#   "gift": true,
#   "delivery_days": 2,
#   "price_value": "It's slightly more expensive than the other leaf blowers out there"
# }

{
  "gift": true,
  "delivery_days": 2,
  "price_value": "It's slightly more expensive than the other leaf blowers out there"
}


In [37]:
type(response.content)

str

In [38]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

### Parse the LLM output string into a Python dictionary

In [39]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [44]:
gift_schema = ResponseSchema(
    name="gift", 
    description="Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.")

delivery_days_schema = ResponseSchema(
    name="delivery_days", 
    description="How many days did it take for the product to arrive? If this information is not found, output -1.")

price_value_schema = ResponseSchema(
    name="price_value", 
    description="Extract any sentences about the value or price, and output them as a comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [45]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [46]:
format_instructions = output_parser.get_format_instructions()

In [47]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.
	"delivery_days": string  // How many days did it take for the product to arrive? If this information is not found, output -1.
	"price_value": string  // Extract any sentences about the value or price, and output them as a comma separated Python list.
}
```


In [48]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [49]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [50]:
response = chat.invoke(messages)

In [51]:
print(response.content)

# ```json
# {
# 	"gift": "True",
# 	"delivery_days": "2",
# 	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
# }
# ```

```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


In [52]:
output_dict = output_parser.parse(response.content)

In [53]:
output_dict

# {'gift': 'True',
#  'delivery_days': '2',
#  'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}

{'gift': 'True',
 'delivery_days': '2',
 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}

In [54]:
type(output_dict)

# dict

dict

In [55]:
output_dict.get('delivery_days')

# '2'

'2'